In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt


In [2]:
classifier_id = 30

In [10]:
stack = 'MD657'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [19]:
request_compute_nodes(16)

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-97ZCW4B5F7TH capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


In [17]:
! tail /home/ubuntu/stderr_1.log

0.50 seconds.
Traceback (most recent call last):
  File "/shared/MouseBrainAtlas/learning/resample_scoremaps_v3.py", line 80, in <module>
    pool.map(resample, range(first_sec, last_sec+1))
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/pool.py", line 251, in map
    return self.map_async(func, iterable, chunksize).get()
  File "/usr/local/lib/python2.7/dist-packages/multiprocess/pool.py", line 567, in get
    raise self._value
IOError: [Errno 2] No such file or directory: '/shared/CSHL_patch_features/Inception-BN/MD657/MD657-F16-2017.02.17-21.02.17_MD657_3_0048_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped/MD657-F16-2017.02.17-21.02.17_MD657_3_0048_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_patch_locations.txt'
Child returned 0


File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt


In [14]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v3.py'),
                'stack': stack,
                'setting': classifier_id},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
               cluster_size=16,
               jobs_per_node=1)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
running svm classifier ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 503.098089 seconds


In [34]:
t = time.time()
sys.stderr.write('Resampling scoremaps ...')

# downscale = 8
downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v3.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

Resampling scoremaps ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 80.844207 seconds


In [35]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
# viz_downscale = 8
viz_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d -d %(downscale)d %(add_label_text)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v3.py'),
                'stack': stack,
                 'setting': classifier_id,
                'add_label_text': '-a' if add_label_text else '',
                'downscale': viz_downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

visualize scoremaps ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-97ZCW4B5F7TH capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 76.672999 seconds


In [38]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

volume_downscale = 32

if stack in all_alt_nissl_ntb_stacks or stack in all_alt_nissl_tracing_stacks:

    run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d -n' % \
                    {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                    'stack': stack,
                    'setting': classifier_id,
                    'downscale': volume_downscale},
                    kwargs_list=dict(structure=all_known_structures),
                    argument_type='single',
                    cluster_size=16)

else:
    
    run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                    'stack': stack,
                    'setting': classifier_id,
                    'downscale': volume_downscale},
                    kwargs_list=dict(structure=all_known_structures),
                    argument_type='single',
                    cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))# 116s

constructing score volumes ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 116.025840 seconds


In [44]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v4.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single',
                cluster_size=16,
               jobs_per_node=1)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# 55 seconds (16 nodes write to respective local /scratch)
# More than 1 simul. processes are not beneficial as they cause local write contention.
# 156 seconds (16 nodes simul. write to /shared)
# 310 seconds (single node, sequential write to /shared)

File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
compute score volume gradients...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 70.765559 seconds


# Global alignment

In [1]:
classifier_id = 2
warp_setting = 1

In [ ]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v3.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting},
#                 kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD591']),
                argument_type='single')

wait_qsub_complete()


sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 1000 seconds ~ 20 mins

In [ ]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_global.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting},
#                 kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD642']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 150 seconds, 300 for with surrounds.

In [ ]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD642']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

In [55]:
! tail /home/ubuntu/stderr_2.log

File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
Gradient LC: 1.565091 seconds
save LC: 46.780748 seconds
